# 01_task1_preprocess_explore.ipynb

# Task 1: Preprocess and Explore Financial Data for Portfolio Optimization

---

## Introduction

In this notebook, we perform Task 1 of the portfolio optimization project for GMF Investments.

The objectives are:
- Download historical adjusted closing price data for TSLA, BND, and SPY from 2015 to 2025.
- Clean and preprocess the data.
- Conduct exploratory data analysis (EDA) to understand trends, volatility, and anomalies.
- Test stationarity using Augmented Dickey-Fuller tests.
- Calculate foundational risk metrics like Value at Risk (VaR) and Sharpe Ratio.

---

## Setup and Imports


In [42]:
from typing import List
import pandas as pd
import yfinance as yf


def load_stock_data(tickers: List[str], start: str, end: str) -> pd.DataFrame:
    """
    Downloads adjusted close price data for given tickers.

    Args:
        tickers (List[str]): List of stock/ETF symbols.
        start (str): Start date in 'YYYY-MM-DD' format.
        end (str): End date in 'YYYY-MM-DD' format.

    Returns:
        pd.DataFrame: Adjusted close prices with tickers as columns.
    """
    # Download historical data
    data = yf.download(tickers, start=start, end=end)

    # Safety check for no data
    if data is None or data.empty:
        raise ValueError(f"No data returned from Yahoo Finance for tickers: {tickers}")

    # Check if data columns are MultiIndex (for multiple tickers)
    if isinstance(data.columns, pd.MultiIndex):
        # Select the 'Adj Close' level data
        if "Adj Close" in data.columns.levels[0]:
            adj_close = data.loc[:, "Adj Close"]
        else:
            raise ValueError("'Adj Close' not found in downloaded data columns.")
    else:
        # Single ticker case, just take 'Adj Close' column
        if "Adj Close" in data.columns:
            adj_close = data["Adj Close"]
        else:
            raise ValueError("'Adj Close' not found in downloaded data columns.")

    # Ensure output is always a DataFrame (for single ticker)
    if isinstance(adj_close, pd.Series):
        adj_close = adj_close.to_frame()

    return adj_close


In [40]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.stattools import adfuller


from src.data_loader import load_stock_data


---

## Load and Inspect Data


In [44]:
import pandas as pd
import yfinance as yf
from typing import List

def load_stock_data(tickers: List[str], start: str, end: str) -> pd.DataFrame:
    """
    Download adjusted close prices for given tickers using auto_adjust.

    Returns a DataFrame with tickers as columns and dates as index.
    """
    data = yf.download(
        tickers,
        start=start,
        end=end,
        progress=False,
        auto_adjust=True,  # <- Adjust prices, so Close = Adj Close
        group_by='ticker',
    )
    
    # When multiple tickers and group_by='ticker', columns are multi-indexed:
    # each ticker is a top-level column, with sub-columns Open, High, Low, Close, Volume.
    # So we extract Close price per ticker.
    if isinstance(data.columns, pd.MultiIndex):
        close_df = pd.DataFrame()
        for ticker in tickers:
            close_df[ticker] = data[ticker]['Close']
    else:
        # Single ticker case
        close_df = data['Close'].to_frame()

    close_df.index.name = 'Date'
    return close_df


In [46]:
# Define tickers and date range
tickers = ["TSLA", "BND", "SPY"]
start_date = "2015-07-01"
end_date = "2025-07-31"

# Load data
adj_close_df = load_stock_data(tickers, start_date, end_date)
adj_close_df.shape

(2535, 3)

---

## Data Cleaning and Preprocessing

- Check for missing values.
- Forward-fill or back-fill missing data if any.
- Calculate daily returns and log returns.


## Install note (if needed)

Run this in your activated `.venv` (Windows PowerShell):

```powershell
pip install yfinance pandas numpy matplotlib seaborn statsmodels jupyter


In [5]:
"""

### Cell 4 (Code)
```python
"""
# Import helper and load data
import sys
sys.path.append("..")  # allow importing src package from notebook

from src.data_loader import prepare_and_save_all, PROCESSED_DIR  # noqa: E402

# parameters
START = "2015-07-01"
END = "2025-07-31"
TICKERS = ["TSLA", "BND", "SPY"]

# download and prepare (will skip if files already exist)
prepare_and_save_all(TICKERS, START, END)

# load processed files into dict
dfs = {}
for t in TICKERS:
    path = PROCESSED_DIR / f"{t}_processed.csv"
    dfs[t] = pd.read_csv(path, parse_dates=["Date"], index_col="Date")
    print(t, dfs[t].shape)


ImportError: cannot import name 'prepare_and_save_all' from 'src.data_loader' (e:\Tenx\Week11\Time Series for Portfolio Optimization\notebooks\..\src\data_loader.py)

## Quick data checks

- Show head/tail
- Data types
- Missing value counts


In [ ]:
for t, df in dfs.items():
    print(f"--- {t} ---")
    display(df.head())
    print("dtypes:")
    print(df.dtypes)
    print("missing values:")
    print(df.isna().sum())
    print()


## Price series — Adj Close / Price

Plot each asset's price over time. Visualize on the same chart
and with log scale for TSLA to show long-term trends.


In [ ]:
plt.figure(figsize=(12, 5))
for t in TICKERS:
    plt.plot(dfs[t]["price"], label=t)
plt.title("Adjusted Close / Price (2015-07-01 → 2025-07-31)")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()

# TSLA log scale
plt.figure(figsize=(12, 4))
plt.plot(dfs["TSLA"]["price"].dropna(), label="TSLA")
plt.yscale("log")
plt.title("TSLA Price (log scale)")
plt.show()


## Daily returns & distributions

Plot daily returns time series and distribution (histogram + KDE).


In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(14, 10))
for i, t in enumerate(TICKERS):
    ax_ts = axes[i, 0]
    ax_dist = axes[i, 1]
    dfs[t]["return"].plot(ax=ax_ts, title=f"{t} daily returns")
    sns.histplot(dfs[t]["return"].dropna(), kde=True, ax=ax_dist)
    ax_dist.set_title(f"{t} return distribution")
plt.tight_layout()
plt.show()


## Rolling statistics — mean & volatility

Compute 21-day rolling mean and annualized rolling volatility
(21-day window ~ 1 month).


In [ ]:
plt.figure(figsize=(14, 8))
for t in TICKERS:
    plt.plot(
        dfs[t]["rolling_mean"].rolling(window=5).mean(),
        label=f"{t} rolling mean (smoothed)",
    )
plt.title("Rolling mean (log returns, smoothed)")
plt.legend()
plt.show()

plt.figure(figsize=(14, 6))
for t in TICKERS:
    plt.plot(dfs[t]["rolling_vol"], label=f"{t} rolling vol (21d, ann)")
plt.title("Rolling annualized volatility (21d)")
plt.legend()
plt.show()


## Outlier detection — extreme return days

List top 5 gains and top 5 losses per ticker.


In [ ]:
for t in TICKERS:
    print(f"--- {t} top 5 gains ---")
    display(dfs[t]["return"].nlargest(5))
    print(f"--- {t} top 5 losses ---")
    display(dfs[t]["return"].nsmallest(5))
    print()


## Stationarity tests (ADF)

We run Augmented Dickey-Fuller on price and log returns.
Interpretation:
- p-value < 0.05 => reject unit root => stationary.
- Price series are typically non-stationary.
- Log returns are often stationary.


In [ ]:
def adf_report(series, name):
    s = series.dropna()
    res = adfuller(s, autolag="AIC")
    stat, pval, usedlag = res[0], res[1], res[2]
    print(f"{name}: ADF stat={stat:.4f}, p-value={pval:.4f}, lag={usedlag}")

for t in TICKERS:
    print(f"=== {t} ===")
    adf_report(dfs[t]["price"], f"{t} price")
    adf_report(dfs[t]["log_return"], f"{t} log return")
    print()


## Risk metrics: VaR (95%) and Annualized Sharpe

- VaR (95%) computed from daily returns historical distribution.
- Sharpe uses mean/std of daily returns annualized by sqrt(252).
  We assume risk-free rate ~ 0 for this task; you can adjust later.


In [ ]:
def var_sharpe(df, p=0.95):
    r = df["return"].dropna()
    var95 = -np.percentile(r, (1 - p) * 100)
    mean = r.mean()
    std = r.std()
    sharpe = (mean / std) * np.sqrt(252) if std != 0 else np.nan
    return {"VaR_95": var95, "mean_daily": mean, "std_daily": std, "sharpe": sharpe}

metrics = []
for t in TICKERS:
    m = var_sharpe(dfs[t])
    m["ticker"] = t
    m["start"] = dfs[t].index.min().date()
    m["end"] = dfs[t].index.max().date()
    metrics.append(m)

metrics_df = pd.DataFrame(metrics).set_index("ticker")
display(metrics_df.round(6))
metrics_df.to_csv("../data/processed/summary_metrics_task1.csv")
print("Saved summary to data/processed/summary_metrics_task1.csv")


## Key insights (example template)

- TSLA shows a long-term upward trend but high volatility.
- BND is stable with low volatility and smaller returns.
- SPY tracks market movements and provides diversification.
- ADF tests typically show non-stationary prices and stationary
  log returns — we will difference prices for ARIMA if needed.


## Next steps (for Task 2)

1. Decide whether to forecast prices, returns, or volatility.
2. Prepare a chronological train/test split.
3. Build and compare ARIMA (statsmodels) and LSTM (PyTorch)
   models.
4. Use forecasted returns for portfolio optimization in Task 4.
